In [5]:
import torch
import torch.nn as nn 
import torch.nn.functional as F
import copy
import math

In [6]:
def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [18]:
class Generator(nn.Module):
    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1)

In [31]:
def Attention(q, k, v):
  d_k = q.size(-1)
  a = torch.matmul(q, torch.transpose(k,1,2))
  a /= math.sqrt(d_k)
  p_attention = a.softmax(dim=-1)
  return torch.matmul(p_attention, v)

In [28]:
a = torch.randn(32,72,512)
print(a[0])

tensor([[-1.1849, -0.8669, -0.9755,  ...,  0.6341, -0.1345, -0.0724],
        [ 0.6982,  1.4099,  0.2470,  ...,  0.6595,  0.0838,  0.6160],
        [ 0.3014, -1.3838,  0.8026,  ..., -1.1056, -0.6107,  1.3418],
        ...,
        [ 2.2372,  0.7165, -0.8512,  ..., -1.7458, -2.3529, -0.1706],
        [-0.5123,  2.1026, -1.3835,  ..., -0.0779, -1.9606, -1.7960],
        [-1.4279, -0.5462,  1.6382,  ..., -1.5663,  0.6481, -1.4983]])


In [8]:
class multihead(nn.Module):
  def __init__(self, h, d_model, dropout=0.1):
    super(multihead, self).__init__()
    assert d_model % h == 0
    self.d_k = d_model // h
    self.h = h
    self.linears = clones(nn.Linear(d_model, d_model), 4)
    self.attn = None
    self.dropout = nn.Dropout(p=dropout)

  def forward(self, q, k, v):
    liste = [q, k, v]
    nbatches = q.size(0)
    for i in range(len(liste)):
      for lin, x in zip(self.linears, liste[i]):
        liste[i] = lin(x)

    q = liste[0]
    k = liste[1]
    v = liste[2]

    x = Attention(q, k, v) 
    return x
      

In [9]:
class FFN(nn.Module):
  def __init__(self, d_model, d_ff):
    super(FFN, self).__init__()
    self.fc1 = nn.Linear(d_model, d_ff)
    self.fc2 = nn.Linear(d_ff, d_model)
  
  def forward(self, x):
    x = F.relu(self.fc1(x))
    x = self.fc2(x)
    return x
  

In [36]:
q = torch.randn(2,3)
k = torch.randn(2,3)
v = torch.randn(2,2)
#x = Attention(q,k,v)
#print(x)
a = torch.randn(32, 72, 512)
b = torch.randn(32, 72, 512)
c = torch.randn(32, 72, 512)
d = Attention(a,b,c)
print(d.size())

torch.Size([32, 72, 512])


In [ ]:
obj = multihead(h=8, d_model=64*8)
print(obj)

multihead(
  (linears): ModuleList(
    (0): Linear(in_features=512, out_features=512, bias=True)
    (1): Linear(in_features=512, out_features=512, bias=True)
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=512, bias=True)
  )
  (dropout): Dropout(p=0.1, inplace=False)
)


In [ ]:
obj2 = FFN(d_model=512, d_ff=2048)
print(obj2)

FFN(
  (fc1): Linear(in_features=512, out_features=2048, bias=True)
  (fc2): Linear(in_features=2048, out_features=512, bias=True)
)


In [10]:
class embeddings(nn.Module):
  def __init__(self, d_model, vocab):
    super(embeddings, self).__init__()
    self.layer = nn.Embedding(vocab, d_model)
    self.d_model = d_model

  def forward(self, x):
    return self.layer(x) * math.sqrt(self.d_model)

In [11]:
class positional_encoding(nn.Module):
  def __init__(self, d_model, dropout=0.1, max_len=5000):
    super(positional_encoding, self).__init__()
    self.dropout = nn.Dropout(p=dropout)
    self.d_model = d_model
    
    pe = torch.zeros(max_len, d_model)

    position = torch.arange(0, max_len).unsqueeze(1)
    div_term = torch.exp(
        torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
    )
    pe[:, 0::2] = torch.sin(position * div_term)
    pe[:, 1::2] = torch.cos(position * div_term)
    pe = pe.unsqueeze(0)
    self.register_buffer("pe", pe)

    def forward(self, x):
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

In [12]:
def rate(step, model_size, factor, warmup):
    if step == 0:
        step = 1
    return factor * (model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5)))

In [15]:
class Layer_Norm(nn.Module):

  def __init__(self, features, eps=1e-6):
    super(Layer_Norm, self).__init__()
    self.a_2 = nn.Parameter(torch.ones(features))
    self.b_2 = nn.Parameter(torch.zeros(features))
    self.eps = eps

  def forward(self, x):
    mean = x.mean(-1, keepdim=True)
    std = x.std(-1, keepdim=True)
    return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [16]:
class Encoding(nn.Module):

  def __init__(self, layer, N):
    super(Encoding, self).__init__()
    self.layers = clones(layer, N)
    self.norm = Layer_Norm(layer.size)

  def forward(self, x):
    for layer in self.layers:
      x = layer(x)
    return self.norm(x)

In [17]:
class Sublayer_Connection(nn.Module):
 
  def __init__(self, size, dropout):
    super(Sublayer_Connection, self).__init__()
    self.dropout = nn.Dropout(p=0.1)
    self.norm = Layer_Norm(size)

  def forward(self, x, sublayer):
    return x + self.dropout(sublayer(self.norm(x)))   